# ConvLSTM

In [4]:
import keras
from keras.models import Sequential
from keras.layers import ConvLSTM2D, Dense, InputLayer, BatchNormalization, Permute

Using TensorFlow backend.
/Users/kibbles/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kibbles/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kibbles/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kibbles/an

Let's just make a model and experiment with it

In [5]:
frames = 24
channels = 5
pixels_x = 21
pixels_y = 21

In [6]:
model = Sequential(name='convlstm_model_0')
model.add(InputLayer(input_shape=(frames, channels, pixels_x, pixels_y)))
model.add(ConvLSTM2D(
    filters=20, kernel_size=(5,5), padding='same', data_format='channels_first',return_sequences=True))
model.add(BatchNormalization(axis=1, ))
model.add(ConvLSTM2D(
    filters=10, kernel_size=(3,3), padding='same', data_format='channels_first',return_sequences=True))
model.add(BatchNormalization(axis=1, ))
model.add(ConvLSTM2D(
    filters=5, kernel_size=(1,1), padding='same', data_format='channels_first',return_sequences=True))
model.add(BatchNormalization(axis=1, ))

model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['mean_absolute_error'])

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model.summary()

Model: "convlstm_model_0"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 24, 20, 21, 21)    50080     
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 20, 21, 21)    96        
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 24, 10, 21, 21)    10840     
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 10, 21, 21)    96        
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 24, 5, 21, 21)     320       
_________________________________________________________________
batch_normalization_3 (Batch (None, 24, 5, 21, 21)     96        
Total params: 61,528
Trainable params: 61,384
Non-trainable params: 144
____________________________________________

## try training

In [22]:
import glob
import xarray as xr
def generate_arrays(img_dir, slice_size=24):
    """
    A generator that returns one 24-hour slice as input, and the subsequent 24-hour slice as output
    """
    # get list of netcdf files in img_dir
    netcdf_dirs = sorted(glob.glob(img_dir+"/*.nc"))
    file_index = 0
    # open first netcdf file
    ds = xr.open_dataset(netcdf_dirs[file_index])
    # counter is for hourly time slices. months with 31 days have 744 hours
    counter = 0
    while True: # generator needs to run infinitely
        
        # get input slice
        input_images = ds.isel( time=slice(counter, counter + slice_size)).to_array().values
        
        # check if we're at the end of the month
        if counter+2*slice_size > ds.sizes['time']:
            # reset slice counter, increment to next netcdf file, open it, get output images
            counter = 0
            file_index += 1
            if file_index == len(netcdf_dirs):
                raise EOFError("End of the line. Last file at ", netcdf_dirs[file_index-1])
                
            ds = xr.open_dataset(netcdf_dirs[file_index])
            # take slice 0-24 as output-image
            output_images = ds.isel( time=slice(counter, counter + slice_size)).to_array().values
            # set counter to -slice_size to reset for input on next iteration
            counter -= slice_size
        # get output slice right after input slice
        else:
            output_images = ds.isel( time=slice(counter+slice_size, counter + 2*slice_size)).to_array().values
        
        # switch frames and channel axes
        input_images = np.moveaxis(input_images, 0, 1)
        output_images = np.moveaxis(output_images, 0, 1)
        # reshape values
        input_images = input_images.reshape(-1, frames, channels, pixels_x, pixels_y)
        output_images = output_images.reshape(-1, frames, channels, pixels_x, pixels_y)
        yield (input_images, output_images)
        counter += slice_size

In [16]:
train_file_path = "../data/train"
validate_file_path = "../data/validate"

In [23]:
history = model.fit_generator(
    generate_arrays(train_file_path),
    steps_per_epoch = 365*3 - 1,
    epochs = 20,
    verbose = 1,
    shuffle = False,
    initial_epoch = 0,
    validation_steps = 365 - 1,
    validation_data = generate_arrays(validate_file_path),
    )

Epoch 1/20
  21/1094 [..............................] - ETA: 13:36 - loss: 1263391.1012 - mean_absolute_error: 19766.7070

KeyboardInterrupt: 

## Save model to ```../models/```

In [24]:
import pickle
import datetime
current_datetime = datetime.datetime.now().strftime("%Y_%M_%d_%H%M")
pickle.dump( history, open( "../models/"+current_datetime+"_convlstm.pkl", "wb" ) )

NameError: name 'history' is not defined

In [ ]:
# #rewrite model in Functional style

# inp = InputLayer(input_shape=(channels, frames, pixels_x, pixels_y))
# permuted = Permute((2,1,3,4))(inp)
# convLstm1 = ConvLSTM2D(filters=20, kernel_size=(5,5), padding='same', 
#                        data_format='channels_first',return_sequences=True)(permuted)
# batch_norm1 = BatchNormalization(axis=1, )(convLstm1)
# convLstm2 = ConvLSTM2D(filters=10, kernel_size=(5,5), padding='same', 
#                        data_format='channels_first',return_sequences=True)(batch_norm1)
# batch_norm2 = BatchNormalization(axis=1, )(convLstm2)
# conv_output = ConvLSTM2D(filters=10, kernel_size=(1,1), padding='same', 
#                        data_format='channels_first',return_sequences=True)(batch_norm2)

# model = Model(inputs = [inp], outputs = [conv_output])
# model.compile(loss='sparse_categorical_crossentropy',
#                   optimizer='adadelta',
#                   metrics=['accuracy'])